In [73]:
import pandas as pd
import nltk

In [74]:
df = pd.read_csv("Tweets.csv")
df = df[["airline_sentiment", "text"]]
df.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [75]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @AmericanAir that's 16+ extra hours of travel ...    2  
neutral                                           @SouthwestAir sent    5  
positive                                            @JetBlue thanks!    5

In [76]:
import nltk
import string
import re

from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('punkt')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [77]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []

    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return(" ".join(y))

In [78]:
print(clean_text("@VirginAmerica What @dhepburn said. fuck off cat"))

@ virginamerica @ dhepburn said . fuck cat


In [79]:
df['text_cleaned'] = df['text'].apply(clean_text)

In [80]:
df.head

<bound method NDFrame.head of       airline_sentiment                                               text  \
0               neutral                @VirginAmerica What @dhepburn said.   
1              positive  @VirginAmerica plus you've added commercials t...   
2               neutral  @VirginAmerica I didn't today... Must mean I n...   
3              negative  @VirginAmerica it's really aggressive to blast...   
4              negative  @VirginAmerica and it's a really big bad thing...   
...                 ...                                                ...   
14635          positive  @AmericanAir thank you we got on a different f...   
14636          negative  @AmericanAir leaving over 20 minutes Late Flig...   
14637           neutral  @AmericanAir Please bring American Airlines to...   
14638          negative  @AmericanAir you have my money, you change my ...   
14639           neutral  @AmericanAir we have 8 ppl so we need 2 know h...   

                                 

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vec = TfidfVectorizer(max_features=3000)
X = tf_vec.fit_transform(df['text_cleaned']).toarray()
Y = df['airline_sentiment'].values
X.shape

(14640, 3000)

In [82]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state = 2)

In [83]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [84]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7219945355191257


In [85]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7493169398907104
